In [1]:
import datetime
import random

import numpy as np
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.applications.resnet50 import ResNet50
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from tqdm import tqdm

from common import *

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
print(".. setting base configuration & params ..")

initialize()

best_model_filepath = './saved-models/achitecture-01.hdf5'
training_epochs = 1
batch_size = 50
images_size = 270


.. setting base configuration & params ..


In [3]:
(train_tensors, X_train, y_train,
test_tensors, X_test, y_test, 
valid_tensors, X_validate, y_validate, 
duration_loading) = load_and_split_data(images_size=images_size)

  0%|          | 0/1408 [00:00<?, ?it/s]

.. loading & splitting data ..


100%|██████████| 352/352 [00:04<00:00, 71.67it/s]


In [5]:
duration_training = train_single_model(model, 
                                       best_model_filepath, 
                                       train_tensors, y_train, 
                                       valid_tensors, y_validate, 
                                       training_epochs, 
                                       batch_size)


.. training the model ..
Train on 1408 samples, validate on 352 samples
Epoch 1/1
1408/1408 [==============================] - 407s 289ms/step - loss: 0.4190 - acc: 0.7876 - val_loss: 0.5799 - val_acc: 0.6761

Epoch 00001: val_loss improved from inf to 0.57987, saving model to ./saved-models/achitecture-01.hdf5


In [7]:
print(".. loading best weights ..")

model.load_weights(best_model_filepath)

.. loading best weights ..


In [9]:
def create_model(optimizer='rmsprop', activation='relu', kernel_size=2):
    model = Sequential()

    model.add(Conv2D(filters=16, kernel_size=kernel_size, padding='same', 
                     input_shape=(270, 270, 3), data_format="channels_last"))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=32, kernel_size=kernel_size, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=kernel_size, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(GlobalAveragePooling2D())
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [10]:
my_classifier = KerasClassifier(build_fn=create_model, verbose=0)

hyperparameters = dict(optimizer=optimizers, 
                       epochs=[5, 10], 
                       batch_size=[5, 10, 100],
                       optimizers=['rmsprop', 'adam'],
                       activation=['relu', 'sigmoid'],
                      )

grid = GridSearchCV(estimator=neural_network, param_grid=hyperparameters)

NameError: name 'optimizers' is not defined

In [ ]:
print(".. testing the model on Test data ..")
section_start_time = datetime.datetime.utcnow()

test_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(test_tensors)]

test_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 1/440 [00:00<01:25,  5.12it/s]

.. testing the model on Test data ..


 31%|███       | 135/440 [00:11<00:25, 11.85it/s]

In [12]:
print(".. testing the model on Training data ..")
section_start_time = datetime.datetime.utcnow()

train_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(train_tensors)]

train_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 1/1408 [00:00<02:44,  8.56it/s]

.. testing the model on Training data ..


100%|██████████| 1408/1408 [01:53<00:00,  8.83it/s]


In [13]:
print(".. testing the model on Validation data ..")
section_start_time = datetime.datetime.utcnow()

validation_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(valid_tensors)]

validation_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 1/352 [00:00<00:44,  7.88it/s]

.. testing the model on Validation data ..


100%|██████████| 352/352 [00:31<00:00, 13.19it/s]


In [20]:
print("________________________________________________________")
print("TIMING SUMMARY:\n")
print("loading duration: {0:.1f} seconds".format(duration_loading))
print("training duration: {0:.1f} seconds".format(duration_training))

print("________________________________________________________")
print("MODEL PERFORMANCE ON TEST DATA:\n")
print("predict duration: {0:.1f} seconds".format(test_data_predictions_time))
false_positive_images, false_negative_images = summarize_model_performance(X_test, y_test, test_data_predictions)

print("________________________________________________________")
print("MODEL PERFORMANCE ON TRAIN DATA:\n")
print("predict duration: {0:.1f} seconds".format(train_data_predictions_time))
_, _ = summarize_model_performance(X_train, y_train, train_data_predictions)

print("________________________________________________________")
print("MODEL PERFORMANCE ON VALIDATION DATA:\n")
print("predict duration: {0:.1f} seconds".format(validation_data_predictions_time))
_, _ = summarize_model_performance(X_validate, y_validate, validation_data_predictions)

________________________________________________________
TIMING SUMMARY:

loading duration: 24.0 seconds
training duration: 23734.4 seconds
________________________________________________________
MODEL PERFORMANCE ON TEST DATA:

predict duration: 34.3 seconds
all:  440
all_positives:  81
all_negatives:  359
true_positives:  73
true_negatives:  335
false_positives:  24
false_negatives:  8
RECALL: 0.90
SPECIFICITY: 0.93
ACCURACY: 0.93
PRECISION: 0.75
F1 SCORE: 0.82
FP RATE / ERROR I: 0.07
FN RATE / ERROR II: 0.10
________________________________________________________
MODEL PERFORMANCE ON TRAIN DATA:

predict duration: 113.8 seconds
all:  1408
all_positives:  316
all_negatives:  1092
true_positives:  277
true_negatives:  1010
false_positives:  82
false_negatives:  39
RECALL: 0.88
SPECIFICITY: 0.92
ACCURACY: 0.91
PRECISION: 0.77
F1 SCORE: 0.82
FP RATE / ERROR I: 0.08
FN RATE / ERROR II: 0.12
________________________________________________________
MODEL PERFORMANCE ON VALIDATION DATA:



## Prior Results

After converting the images to grayscale:

```
________________________________________________________
MODEL PERFORMANCE ON TEST DATA:

predict duration: 33.6 seconds
all:  440
all_positives:  81
all_negatives:  359
true_positives:  56
true_negatives:  342
false_positives:  17
false_negatives:  25
RECALL: 0.69
SPECIFICITY: 0.95
ACCURACY: 0.90
PRECISION: 0.77
F1 SCORE: 0.73
FP RATE / ERROR I: 0.05
FN RATE / ERROR II: 0.31
________________________________________________________
MODEL PERFORMANCE ON TRAIN DATA:

predict duration: 104.4 seconds
all:  1408
all_positives:  316
all_negatives:  1092
true_positives:  219
true_negatives:  1022
false_positives:  70
false_negatives:  97
RECALL: 0.69
SPECIFICITY: 0.94
ACCURACY: 0.88
PRECISION: 0.76
F1 SCORE: 0.72
FP RATE / ERROR I: 0.06
FN RATE / ERROR II: 0.31
________________________________________________________
MODEL PERFORMANCE ON VALIDATION DATA:

predict duration: 24.3 seconds
all:  352
all_positives:  66
all_negatives:  286
true_positives:  49
true_negatives:  273
false_positives:  13
false_negatives:  17
RECALL: 0.74
SPECIFICITY: 0.95
ACCURACY: 0.91
PRECISION: 0.79
F1 SCORE: 0.77
FP RATE / ERROR I: 0.05
FN RATE / ERROR II: 0.26
```

The 90_93 model:
```
________________________________________________________
TIMING SUMMARY:

loading duration: 24.0 seconds
training duration: 23734.4 seconds
________________________________________________________
MODEL PERFORMANCE ON TEST DATA:

predict duration: 34.3 seconds
all:  440
all_positives:  81
all_negatives:  359
true_positives:  73
true_negatives:  335
false_positives:  24
false_negatives:  8
RECALL: 0.90
SPECIFICITY: 0.93
ACCURACY: 0.93
PRECISION: 0.75
F1 SCORE: 0.82
FP RATE / ERROR I: 0.07
FN RATE / ERROR II: 0.10
________________________________________________________
MODEL PERFORMANCE ON TRAIN DATA:

predict duration: 113.8 seconds
all:  1408
all_positives:  316
all_negatives:  1092
true_positives:  277
true_negatives:  1010
false_positives:  82
false_negatives:  39
RECALL: 0.88
SPECIFICITY: 0.92
ACCURACY: 0.91
PRECISION: 0.77
F1 SCORE: 0.82
FP RATE / ERROR I: 0.08
FN RATE / ERROR II: 0.12
________________________________________________________
MODEL PERFORMANCE ON VALIDATION DATA:

predict duration: 31.4 seconds
all:  352
all_positives:  66
all_negatives:  286
true_positives:  58
true_negatives:  268
false_positives:  18
false_negatives:  8
RECALL: 0.88
SPECIFICITY: 0.94
ACCURACY: 0.93
PRECISION: 0.76
F1 SCORE: 0.82
FP RATE / ERROR I: 0.06
FN RATE / ERROR II: 0.12
```

## False Positives (first 20):

In [21]:
display_images(false_positive_images[:20])

## False Negatives (first 20):

In [22]:
display_images(false_negative_images[:20])